In [34]:
from glob import glob
import shutil
import os
from tqdm import tqdm
from zipfile import ZipFile
import sys
import pandas as pd

# 1. Download KTH Dataset from URL 

In [26]:
target_path = '/NOBACKUP/gauthiers/test'
dataset_path = '/NOBACKUP/gauthiers/kymatio_mod/covidx-cxr2.zip'

In [27]:
os.path.join(target_path, 'xray')

'/NOBACKUP/gauthiers/test/xray'

# 2. Extract .tar and delete .tar

In [28]:
# Open your .zip file
with ZipFile(file=dataset_path) as zip_file:
    for file in tqdm(iterable=zip_file.namelist(), total=len(zip_file.namelist())):
        zip_file.extract(member=file, path=os.path.join(target_path, 'xray'))
os.remove(dataset_path)

100%|██████████| 16753/16753 [05:36<00:00, 49.82it/s] 


# 3. Create Dataset


In [33]:
folders = glob(f"{target_path}/xray/*")
folders

['/NOBACKUP/gauthiers/test/xray/train.txt',
 '/NOBACKUP/gauthiers/test/xray/train',
 '/NOBACKUP/gauthiers/test/xray/test',
 '/NOBACKUP/gauthiers/test/xray/test.txt',
 '/NOBACKUP/gauthiers/test/xray/competition_test']

## Create Train Folder

In [57]:
df_train = pd.read_csv(os.path.join(target_path, 'xray', 'train.txt'), delimiter=' ',
                                        header = 0 )
df_test = pd.read_csv(os.path.join(target_path, 'xray', 'test.txt'), delimiter=' ', header = 0)
df_train.columns=['patient_id', 'filename', 'class', 'data_source']
df_test.columns=['patient_id', 'filename', 'class', 'data_source']
df_train.head(5)

,patient_id,filename,class,data_source
0,25,acute-respiratory-distress-syndrome-ards-1.jpg,negative,cohen
1,26,acute-respiratory-distress-syndrome-ards.jpg,negative,cohen
2,27,ards-secondary-to-tiger-snake-bite.png,negative,cohen
3,28,pneumocystis-pneumonia-2-PA.png,negative,cohen
4,38,F051E018-DAD1-4506-AD43-BE4CA29E960B.jpeg,negative,cohen


In [74]:
%%time
# iterate over rows in train dataframe
folder_path = os.path.join(target_path, 'xray_preprocess/train')
for index, row in tqdm(df_train.iterrows(), total=df_train.shape[0]):
    if row['class']=='negative':
        destination_path = os.path.join(folder_path, 'negative')
    elif row['class']=='positive':
        destination_path = os.path.join(folder_path, 'positive')
    if not os.path.exists(destination_path):
        os.makedirs(destination_path) 
    img = os.path.join(target_path, 'xray', 'train', row['filename'])
    shutil.copy(img, destination_path )


100%|██████████| 15950/15950 [03:27<00:00, 77.04it/s] 

CPU times: user 12.8 s, sys: 22.5 s, total: 35.3 s
Wall time: 3min 27s


In [76]:
%%time
# iterate over rows in test dataframe
folder_path = os.path.join(target_path, 'xray_preprocess/test')
for index, row in tqdm(df_test.iterrows(), total=df_test.shape[0]):
    if row['class']=='negative':
        destination_path = os.path.join(folder_path, 'negative')
    elif row['class']=='positive':
        destination_path = os.path.join(folder_path, 'positive')
    if not os.path.exists(destination_path):
        os.makedirs(destination_path) 
    img = os.path.join(target_path, 'xray', 'test', row['filename'])
    shutil.copy(img, destination_path )

100%|██████████| 399/399 [00:11<00:00, 34.37it/s] 

CPU times: user 648 ms, sys: 1.76 s, total: 2.41 s
Wall time: 11.6 s
